In [ ]:
WORKSPACE = "/workspace"
YAML = WORKSPACE + "/benchmark_config.yaml"
BASE_DATASTORE = WORKSPACE + "/base.datastore.json"
TEST_DATASTORE = WORKSPACE + "/test.datastore.json"
BASE_METADATA = WORKSPACE + "/base.testrun_metadata.json"
TEST_METADATA = WORKSPACE + "/test.testrun_metadata.json"

BASE_TESTRUN_RESULT = WORKSPACE + "/base.testrun_result.csv"
TEST_TESTRUN_RESULT = WORKSPACE + "/test.testrun_result.csv"
METADATA = WORKSPACE + "/2way_metadata.csv"
BENCHMARK = WORKSPACE + "/2way_benchmark.csv"

import os
import sys
import yaml
import json
import pandas as pd
from IPython.display import display, HTML, Markdown
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)
from datetime import datetime

BASEPATH = os.path.abspath('.')
SCRIPTPATH = BASEPATH + "/../fetch_scripts"

In [ ]:
# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)

# Performance Test Report Portal

In [ ]:
now = datetime.now()
dt_string = "Generate time: *{}*".format(now.strftime("%Y-%m-%d %H:%M:%S"))
display(Markdown(dt_string))

## Introduction

In [ ]:
def read_json(json_file):
    with open(json_file, 'r') as f:
        try:
            data = json.load(f)
        except Exception as e:
            print("Fail to load {}".format(json_file))
            raise
    return data

base_metadata = read_json(BASE_METADATA)
test_metadata = read_json(TEST_METADATA)
assert base_metadata.get("testrun.type") == test_metadata.get("testrun.type"), "Base and Test type must be the same! Exit."

run_type = base_metadata.get("testrun.type")
base_platform = base_metadata.get("testrun.platform")
test_platform = test_metadata.get("testrun.platform")
base_id = base_metadata.get("testrun.id")
test_id = test_metadata.get("testrun.id")
# Type and platform must not be None
assert run_type is not None, "Type is None! Exit."
assert base_platform is not None, "Base platform is None! Exit."
assert test_platform is not None, "Test platform is None! Exit."

with open('{}/templates/{}_{}.md'.format(BASEPATH, base_platform.lower(), run_type), 'r') as f:
    display(Markdown(f.read()))
    
if base_platform != test_platform:
    with open('{}/templates/{}_{}.md'.format(BASEPATH, test_platform.lower(), run_type), 'r') as f:
        display(Markdown('\n'+f.read()))

In [ ]:
%%html
<style>
    table {
        display: inline-block
    }
</style>

In [ ]:
# Generate Base testrun result script
base_gen_result_script = "{}/generate_testrun_results.py --config {} --datastore {} --metadata {} --output {}".format(
SCRIPTPATH, YAML, BASE_DATASTORE, BASE_METADATA, BASE_TESTRUN_RESULT)

# Generate Test testrun result script
test_gen_result_script = "{}/generate_testrun_results.py --config {} --datastore {} --metadata {} --output {}".format(
SCRIPTPATH, YAML, TEST_DATASTORE, TEST_METADATA, TEST_TESTRUN_RESULT)

# Generate 2way metadata script
gen_metadata_script = "{}/generate_2way_metadata.py --test {} --base {} --output {}".format(
SCRIPTPATH, TEST_METADATA, BASE_METADATA, METADATA)

# Generate 2way benchmark script
gen_benchmark_script = "{}/generate_2way_benchmark.py --config {} --test {} --base {} --output {}".format(
SCRIPTPATH, YAML, TEST_TESTRUN_RESULT, BASE_TESTRUN_RESULT, BENCHMARK)

# Run scripts parallelly
import multiprocessing
all_processes = (base_gen_result_script, test_gen_result_script, gen_metadata_script)   

def execute(process):                                                             
    os.system(f'python3 {process}') 

process_pool = multiprocessing.Pool(processes = 3)                                                        
process_pool.map(execute, all_processes)

for result in [BASE_TESTRUN_RESULT, TEST_TESTRUN_RESULT, METADATA]:
    assert os.path.exists(result), "Fail to generate {}! Exit.".format(result)

# Generate 2way benchmark
os.system('python3 {}'.format(gen_benchmark_script))
assert os.path.exists(BENCHMARK), "Fail to generate {}! Exit.".format(BENCHMARK)

In [ ]:
def color_delta(val):
    color_dict = {
        "Major Regression": 'red',
        "Minor Regression": 'black',
        "Major Improvement": 'green',
        "Minor Improvement": 'black',
        "Variance Too Large": 'orange',
        "No Significance": 'black',
        
    }
    return 'color: {}'.format(color_dict.get(val, 'black'))

def highlight_cols(s):
    return 'background-color: #eeffff'

def bold_font(s):
    return 'font-weight: bold'

def displayComparison(df):
    #These are the columns which need special formatting
    deltacols=df.columns.map(lambda x: x.endswith("-SPEC"))
#    roundcols=df.columns.map(lambda x: x.endswith(("-AVG", "", "-%SD", "-%DIFF", "-SIGN")))
    display(df.style.applymap(color_delta,subset=deltacols).applymap(bold_font,subset=deltacols))
#    display(df.style.applymap(color_delta,subset=deltacols).applymap(bold_font,subset=deltacols).format(FORMATER, subset=roundcols).hide_index())

## Metadata

In [ ]:
%%HTML
* The differences between Test and Base are <b style='color:orange'>highlighted</b>.

In [ ]:
def highlight_diff(row, cell_format):
    cell_format = cell_format if row['TEST'] != row['BASE'] else ''
    format_row = ['', cell_format, cell_format]
    return format_row

def color_diff(row):
    return highlight_diff(row, 'color: orange')

def bold_diff(row):
    return highlight_diff(row, 'font-weight: bold')
    
conf_df = pd.read_csv(METADATA, index_col=0)
conf_df.fillna('', inplace=True)
#conf_df = conf_df[['KEY', 'TEST', 'BASE']]
#sorter = ['testrun.id'] + [x for x in conf_df['KEY'] if x != 'testrun.id']
#conf_df['KEY'] = conf_df['KEY'].astype("category")
#conf_df["KEY"].cat.set_categories(sorter, inplace=True)
#conf_df.sort_values(['KEY'], inplace=True)
display(conf_df.style.applymap(bold_font, subset=['KEY']).apply(color_diff, axis=1).apply(bold_diff, axis=1).hide_index())

## Summary

In [ ]:
with open('{}/templates/summary_introduction.html'.format(BASEPATH), 'r') as f:
    display(HTML(f.read()))

In [ ]:
benchmark_df = pd.read_csv(BENCHMARK, index_col=0, dtype = str, keep_default_na=False)
summary_df = benchmark_df[['RW','BS','IOdepth','Numjobs']+list(benchmark_df.filter(regex='-SPEC$').columns)]
displayComparison(summary_df)

## Details

This section shows the detail data of benchmark report, base run result and test run result. 

### Detail benchmark report

In [ ]:
displayComparison(benchmark_df)

In [ ]:
# Convert testrun result Path column value to link
def formatRunResult(df, testrun_id):
    # Convert Path to link and round data
    display(df.style.format({'Path': lambda x: '<a target="_blank" href="../../testruns/{}/{}">raw data</a>'.format(testrun_id, x)}))
#    display(df.style.format({'Path': make_clickable}).format(FORMATER, subset=['IOPS', 'LAT(ms)', 'CLAT(ms)']))

### Base run result

In [ ]:
base_df = pd.read_csv(BASE_TESTRUN_RESULT, index_col=0, dtype = str)
formatRunResult(base_df, base_id)

### Test run result

In [ ]:
test_df = pd.read_csv(TEST_TESTRUN_RESULT, index_col=0)
formatRunResult(test_df, test_id)